# 两个原则
> 1. 平移不变性
> 2. 局部性

# 重新考虑全连接层
> 1. 将输入和输出变形为矩阵(宽度，高度)
> 2. 将权重变形为4-D张量(h, w)到(h', w')
<img alt="image.png" height="300" src="/Users/loneker/Desktop/d2l/pic/iShot_2024-04-23_15.48.52.png" width="700"/>

# 总结
> 1. 卷积层将输入和核矩阵进行交叉相关，加上偏移后得到输出
> 2. 核矩阵和偏移是可学习的参数
> 3. 核矩阵的大小是超参数

# 实现二维互相关运算

In [1]:
import torch
from torch import nn
from d2l import torch as d2l

# 计算二维互相关运算， X是输入， K是核矩阵
def corr2d(X, K) : 
    h, w = K.shape
    # Y就是输出的高度和宽度
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]) :
        for j in range(Y.shape[1]) : 
            Y[i, j] = (X[i : i + h, j : j + w] * K).sum()
    return Y

In [2]:
# 验证二维互相关运算的输出
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

# 实现二维卷积层

In [3]:
class Conv2D(nn.Module) : 
    def __init__(self, kernel_size):
        super.__init__()
        # weight和bias是可学习参数
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))
        
    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

# 卷积层的简单应用： 检测图像中不同颜色的边缘

In [4]:
X = torch.ones((6, 8))
X[:, 2 : 6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [5]:
K = torch.tensor([[1.0, -1.0]]) # 如果横向连续两个数一样，则结果为0，否则不为0
Y= corr2d(X, K)
Y   # Y 中的 1 代表从白色到黑色的边缘， -1 代表从黑色到白色的边缘

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

# 学习由X生成Y的卷积核(现在不知道K是什么样子)

In [10]:
conv2d = nn.Conv2d(1, 1, kernel_size=(1, 2), bias = False)

X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))

for i in range(10) : 
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    # 手动实现优化器
    conv2d.weight.data[:] -= 3e-2 * conv2d.weight.grad
    if (i + 1) % 2 == 0 :
        print(f'batch {i + 1}, loss {l.sum() : .3f}')

batch 2, loss  6.565
batch 4, loss  1.135
batch 6, loss  0.204
batch 8, loss  0.040
batch 10, loss  0.009


In [11]:
conv2d.weight.data.reshape((1, 2))

tensor([[ 0.9804, -0.9932]])